In [ ]:
# import settings and functions
%run ./../../../imports.ipynb


In [ ]:
# regular sparse
prefix = "c4_brick_regular_sparse_"
csv_name = 'dataset_regular_sparse_scaled.csv'

# # regular semidense
# prefix = "c4_brick_regular_semidense_"
# csv_name = 'dataset_regular_semidense_scaled.csv'

# # regular semidense
# prefix = "c4_brick_regular_dense_"
# csv_name = 'dataset_regular_dense_scaled.csv'

# # random noisy
# prefix = "c4_brick_random_noisy_"
# csv_name = 'dataset_random_dense_noisy_scaled.csv'


run_test = True
run_analysis = True

run_test = False
run_analysis = False

color_lims = True
# color_lims = False

In [ ]:

!cp ./../../../datasets_brick/scaling.in .
!cp ./../../../datasets_brick/regular/{csv_name} .

scal = np.genfromtxt('./scaling.in')
scaling = scal
print(scaling)

## Analysis setup

In [ ]:
# which executable?

exe = data_driven_diffusion_snes
# exe = hdiv_data_driven_diffusion_snes
sumanalys = "sumanalys.csv"
ana_name = "ana_DD_brick"

ana_compare_exe = [data_driven_diffusion_snes]
ana_compare_name = ["ana_DD_brick"]
ana_compare_sum = ["sumanalys.csv"]

# ana_compare_exe = [hdiv_diffusion, classic_diffusion]
# ana_compare_name = ["ana_brick_mixed", "ana_brick_classic"]
# ana_compare_sum = ["sumanalys.csv", "FEM_errors.csv"]

# Convergence analysis parameters
order_list = [1, 2, 3] # approximation order p
# elem_size_list = [0.5, 0.2, 0.1, 0.05, 0.02] # element size h
# order_list = [1, 2, 3] # approximation order p
# elem_size_list = [0.5, 0.2, 0.1, 0.05] # element size h
params.triangle_mesh = True
params.nproc = 1 # number of processors
jumps = ""
if params.nproc == 1:
    jumps = "-get_jumps"
# jumps = "-get_jumps"

naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

In [ ]:
params.conductivity = 1.0 # linear conductivity
params.element_size = 0.2 # element size in the regular mesh
# params.element_size = elem_size_list[0] # element size in the regular mesh
params.order = 3 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
params.mesh_file = "L_shape"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 


### New brick meshes

In [ ]:
params.list_mesh_files = []
params.element_sizes = []

params.list_mesh_files.append("./../brick_20.cub")
params.element_sizes.append(5)

params.list_mesh_files.append("./../brick_15.cub")
params.element_sizes.append(4)

params.list_mesh_files.append("./../brick_10.cub")
params.element_sizes.append(3)

params.list_mesh_files.append("./../brick_7.cub")
params.element_sizes.append(2)

params.list_mesh_files.append("./../brick_6.cub")
params.element_sizes.append(1)

params.list_mesh_files.append("./../brick_4.cub")
params.element_sizes.append(1)

# which mesh to use
input_number = 3

In [ ]:
h = params.element_sizes
cub_names = params.list_mesh_files
input_names = []
for i in cub_names:
    input_names.append(i[:-3]+"h5m")
count = len(h)
print(cub_names)
print(input_names)

In [ ]:
# # %%capture
# # partition the mesh
# for i in range(count):
#     # print(i)
#     !{tools_dir}/mofem_part -my_file {cub_names[i]} -output_file {input_names[i]} -my_nparts 1 -dim 2 -adj_dim 1

## Run test

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
# # Testing mesh generation
# if run_test:
#     params.show_mesh = True
#     generate_config(params)
#     generate_mesh(params)

In [ ]:
# -csv_tree_file data_VGQ_generated_scaled.csv

params.csv_tree_file = csv_name

In [ ]:

# a = 134.0
# b = -0.1047
# c = 3.719 *10**(-5)

# nonlinear_a = a
# nonlinear_b = b
# nonlinear_c = c


# if run_test:
#     !rm out*
#     params.part_file = input_names[0]
#     !{diffusion_nonlinear_graphite} -file_name {params.part_file} -my_order 2 -nonlinear_a {nonlinear_a} -nonlinear_b {nonlinear_b} -nonlinear_c {nonlinear_c} -scaling 0
#     !convert.py out*

In [ ]:
# Testing running analysis


# exe = hdiv_diffusion
# # exe = classic_diffusion
# # exe = data_driven_diffusion
# # # exe = data_driven_diffusion_snes
# exe = hdiv_data_driven_diffusion_snes
# # exe = hdiv_data_driven_diffusion

if run_test:
    !rm out*[0-9]*
    params.part_file = input_names[3]
    # params.mesh_file = cub_names[0]
    # params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    # !{mofem_part} -my_file {params.mesh_file} -nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    !mpirun -np {params.nproc} {exe} -file_name {params.part_file} -my_order {params.order} {jumps} -csv_tree_file {params.csv_tree_file} -scaling 1 -snes_max_it 500 -print_integ
    #  -use_line -data_dim 4 -my_dummy_k 100.0



In [ ]:
name_vtk_integ = "out_integ_pts_"+csv_name[:-4]
name_vtk_result = "out_result_"+csv_name[:-4]
name_vtk_error = "out_error_"+csv_name[:-4]
name_vtk_moab = "out_moab_"+csv_name[:-4]

print(name_vtk_integ)

In [ ]:
if run_test:    
    out_to_vtk = !ls -c1 out_integ_pts_*.h5m
    last_file=out_to_vtk[0]
    !mbconvert {last_file} {name_vtk_integ}.vtk
    !mbconvert out_result_0.h5m {name_vtk_result}.vtk
    !mbconvert out_error_1000.h5m {name_vtk_error}.vtk
    # !mbconvert out_moab_5.h5m {name_vtk_moab}.vtk
    !rm out_*h5m


In [ ]:
# import settings and functions
%run ./../plot_for_DD.ipynb